# Pràctica 6: extracció d'informació
En aquesta pràctica analitzarem de manera no supervisada un corpus de textos per a extraure informació.\
Primer extraurem i analitzarem les entitats que formen el corpus, a continuació extraurem les paraules claus de cada document i finalment realitzarem un *topic *modeling* del corpus.\
Usem el conjunt de crítiques de cinema de "Mundocine" que estan en format XML dins d'un directori.  
Definim una funció per a extraure el text de la crítica de cada arxiu XML del directori mitjançant una funció de tipus **generator*

In [2]:
import os, re
from xml.dom.minidom import parseString

def parse_folder(path):
    """generator that reads the contents of XML files in a folder
    Returns the <body> of the <review> in each XML file.
    XML files encoded as 'latin-1'"""
    for file in sorted([f for f in os.listdir(path) if f.endswith('.xml')],
                        key=lambda x: int(re.match(r'\d+',x).group())):
        with open(os.path.join(path, file), encoding='latin-1') as f:
            doc=parseString(re.sub(r'(<>)|&|(<-)', '', f.read()))

            titulo = doc.documentElement.attributes["title"].value

            btxt = ""
            review_bod = doc.getElementsByTagName("body")
            if len(review_bod) > 0:
                for node in review_bod[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        btxt += node.data + " "

            rtxt = ""
            review_summ = doc.getElementsByTagName("summary")
            if len(review_summ) > 0:
                for node in review_summ[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        rtxt += node.data + " "
                #separamos después de ciertos signos de puntuación
                rtxt = re.sub(r"([\.\?])", r"\1 ", rtxt)
                        
            rank = int(doc.documentElement.attributes["rank"].value)
            
            yield titulo, rtxt, btxt, rank


## Extracció d'entitats
Analitzarem les entitats (tipus i quantitat) que apareixen en cada document del *corpus*
### Exercici
Construeix una funció de tipus *generator* que retorne en cada iteració les entitats del següent document. Aquestes entitats es generaran com un *string* amb les etiquetes de cada entitat en el text separat per comes, p. ej:
```
'MISC PER MISC MISC MISC MISC ORG PER PER MISC LOC'
```

In [3]:
import spacy

nlp = spacy.load("es_core_news_sm")

def extraer_ner(texto):
    doc = nlp(texto)
    ent_labels = [ent.label_ for ent in doc.ents]
    return ' '.join(ent_labels)

Prova el seu funcionament sobre el text de la primera crítica en el Corpus.

In [4]:
corpus = parse_folder('criticas/train/')
texto = extraer_ner(next(corpus)[2])
texto


'MISC MISC MISC PER PER MISC MISC MISC MISC MISC MISC PER PER PER MISC LOC PER MISC MISC MISC MISC PER MISC'

Combinarem totes dues funcions per a processar tot el corpus.\
Construeix una funció de tipus *generator* que a partir d'un directori amb les crítiques, processe tots els seus arxius i retorne en cada iteració el llistat d'entitats del següent document

In [5]:
def criticas_ner(folder):
    corpus = parse_folder(folder)
    for critica in corpus:
        yield extraer_ner(critica[2])

Provem el seu funcionament amb el primer arxiu:

In [6]:
criticas_gen = criticas_ner("criticas/train")
next(criticas_gen)

'MISC MISC MISC PER PER MISC MISC MISC MISC MISC MISC PER PER PER MISC LOC PER MISC MISC MISC MISC PER MISC'

Una vegada s'ha consumit un arxiu, el generador passa el següent i no es pot tornar a l'inici del iterador:

In [7]:
next(criticas_gen)

'MISC MISC MISC MISC MISC LOC MISC PER MISC PER LOC MISC LOC PER PER MISC PER MISC PER MISC MISC LOC PER PER PER MISC MISC MISC MISC MISC PER PER MISC PER PER PER LOC'

## Anàlisi de les entitats
Com a anàlisi molt simple, veurem quantes entitats apareixen en tot el corpus.\
Per a això, comptem el núm. d'entitats de cada document amb el vectorizador BoW de la llibreria `scikit-learn`.
### Exercici
Aplica el vectorizador BoW per a obtindre la seua matriu d'ocurrència. Suma el total de vegades que ha aparegut cada entitat i mostra'l en un dataframe de Colles.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vect = CountVectorizer()
bow = vect.fit_transform(criticas_gen)
totales = bow.sum(axis=0)

data = {'Entidad' : vect.get_feature_names_out(), 'Apariciones' : totales.tolist()[0] }

df = pd.DataFrame(data)
df

,Entidad,Apariciones
0,loc,10646
1,misc,32718
2,org,2812
3,per,29328


## Extracció de paraules clau
En aquest exercici extraurem les paraules clau de cada crítica mitjançant la llibreria `textacy`. Després crearem un BoW d'aquests termes per a analitzar amb quins freqüència apareixen.\
Com les paraules claus determinades per la llibreria `textacy` poden ser n-grames, cal unificar els seus constituents amb `_` per a considerar-los termes únics en el vocabulari.  

### Exercici
Defineix una funció per a unir diversos n-*gramas en un únic terme.\
Defineix una funció que extraga les paraules clau d'un document i les retorne com una llista de *tokens*. Utilitza l'algorisme `textrank` sobre el text normalitzat en minúscules sense lematizar.

In [14]:
from textacy.extract import keyterms as kt

def unir_ngramas(texto):
    """Une todos los términos de un n-grama mediante '_'
    para que formen un único término en el vocabulario"""
    #No se que coño tengo que hacer aqui??????????????

def extraer_keywords(texto, topn=10):
    """Extrar las palabras clave de un texto mediante la
    librería textacy
    Devuelve los topn términos clave como lista de strings"""
    # Y esto no se si esta bien ?????????????
    keywords = kt.sgrank(nlp(texto), ngrams=(1, 2, 3), normalize="lower", include_pos=["NOUN", "PROPN", "ADJ"])
    return [keyword for keyword in keywords[:topn]]

Provem sobre la crítica descarregada abans...

In [25]:
corpus = parse_folder('criticas/train/')
unir_ngramas(next(corpus)[2])

AttributeError: module 'textacy.extract.keyterms' has no attribute 'ngrams'

### Exercici
Defineix una funció *generator* que retorne iterativament per a cada crítica dins d'un directori les seues paraules clau.
Ha de retornar per cada document un *string* amb les paraules clau separades per espai, per a poder usar-ho amb el vectorizador de `scikit-learn`

In [11]:
def criticas_keyword(folder):
    #COMPLETAR

SyntaxError: incomplete input (3725003253.py, line 2)

Provem sobre el primer document del corpus...

In [ ]:
#COMPLETAR

### Anàlisi de les paraules clau
Implementa un vectorizador BoW per a comptar les paraules clau que apareixen almenys en un 0,5% dels documents i mostra-les ordenades en ordre descendent de freqüència (paraula, núm. d'aparicions en total) dins d'un *DataFrame. Mostra només els 10 termes més utilitzats.

In [ ]:
#COMPLETAR

### Exercici
També ho podem fer amb la llibreria `gensim`, per a així poder comptar el núm. de documents en el qual apareix cada keyword (amb l'atribut `dfs` de l'objecte `Dictionary`).\
Defineix de nou la funció `critiques_keyword` perquè retorne les paraules clau de cada document en un format compatible amb Gensim i calcula el seu diccionari BoW (no fa falta calcular la matriu de tots els documents).\
Després, utilitza el mètode `filter_extremes` per a quedar-te amb els termes que apareixen almenys en un 0,5% dels documents (nota: usa l'atribut `.num_docs` per a calcular el núm. de documents que formen el 0,5% del total)\
Finalment, usa l'atribut `dfs` del diccionari per a mostrar les paraules clau i el núm. de documents en el qual s'usen, com *dataframe. Mostra només els 10 termes més freqüents.

In [ ]:
#COMPLETAR

## Topic modeling
Finalment, calcularem les temàtiques del corpus de crítiques mitjançant un model de *topic modeling* usant l'algorisme LDA.

In [ ]:
# Gensim
import gensim
from gensim.models import LdaModel
from pprint import pprint
import warnings


# herramientas de dibujado
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

Definim una classe *iterable* per a obtindre els documents del Corpus línia a línia des de l'arxiu del conjunt d'exemple i convertir-los en un llistat de tokens. Al contrari que els generadors, les classes *iterables* poden tornar a l'inici de la llista cada vegada, i no s'esgoten quan es consumeixen tots els elements.
### Exercici
Per a calcular les temàtiques, utilitzarem els lemes de cada terme del corpus, considerant només aquells la funció morfològica dels quals siga nom, nom propi, adjectiu, verb o adverbi. Defineix una funció `lematize_doc` que retorne per a cada document un llistat dels tokens amb aquest processament.

In [ ]:
def lemmatize_doc(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    """Función que devuelve la lista de lemas en minúscula de una string,,
    excluyendo las palabras cuyo POS_TAG no está en la lista allowed_postags.
    Considera sólo lemas de más de 3 caracteres y omite las stop words."""
    #COMPLETAR

def criticas_tokens(folder):
    for c in parse_folder(folder):
        yield lemmatize_doc(c[2])
            
#para no tener que cargar todo el corpus en memoria creamos un streamer
class BOW_Corpus(object):
    """
    Iterable: en cada iteración devuelve el vector bag-of-words
    del siguiente documento en el corpus.
    El corpus es el listado de críticas alojadas en el directorio
    pasado como argumento al instanciar la clase.
    
    Procesa un documento cada vez, así
    nunca carga el corpus entero en RAM.
    """
    def __init__(self, dirname):
        self.dirname = dirname
        #crea el diccionario = mapeo de documentos a sparse vectors
        self.diccionario = gensim.corpora.Dictionary(criticas_tokens(self.dirname))
        
    def __len__(self):
        #necesitamos saber la longitud del corpus para visualizar con pyLDAvis
        return self.diccionario.num_docs
    
    def __iter__(self):
        """
        __iter__ es un iterable => BOW_Corpus es un streamed iterable.
        """
        for tokens in criticas_tokens(self.dirname):
            # transforma cada doc (lista de tokens) en un vector sparse uno a uno
            yield self.diccionario.doc2bow(tokens)

Crea la matriu BoW sobre el corpus com un element de la classe BOW_Corpus amb el nom `bow_critiques`

In [ ]:
#COMPLETAR

Mostrem el BoW del primer document com a comprovació

In [ ]:
for b in bow_criticas:
    print(b)
    break

In [ ]:
#términos en el diccionario
len(bow_criticas.diccionario.token2id)

In [ ]:
#longitud del corpus
len(bow_criticas)

### Exercici
Crea un model LDA sobre el corpus de crítiques amb 5 temes i mostra'l gràficament usant les funcions de la llibreria `pyLDAvis`

In [ ]:
#COMPLETAR